In [22]:
import os
import json
from ipyleaflet import Map, GeoData, basemaps, LayersControl, WidgetControl
from ipywidgets import Textarea
import geopandas as gpd

In [23]:
features_gdf = gpd.read_file("./download/Leveed_Areas.shp")
boundary_gdf = gpd.read_file("./boundary/FHP_Ohio_River_Basin_boundary.shp")

features_gdf.head()
#gdf_original.hvplot(global_extent=True, frame_height=450, tiles=True)

,OBJECTID,LEVEED_ID,LEVEE_STAT,FEATURE_NA,MIN_OVERTO,EGRESS_NUM,COMPUTED_S,Shape__Are,Shape__Len,COMPUTED_1,...,RESPONSIBL,PROJECT_DE,RISK_CHAR_,P2_NUMBERS,AVERAGE_HE,SYSTEM_SUM,PROJ_DESC_,RISK_CHA_1,LAST_UPDAT,geometry
0,1,1806000068,None,West side Madison River South of I-90 N segment,None,NaN,None,0.000005,0.014352,None,...,Three Forks Dyke District,None,None,None,None,None,None,None,None,"POLYGON Z ((-111.52498 45.89630 0.00000, -111...."
1,2,1806000067,None,West side Madison River North of I-90,None,NaN,None,0.000049,0.044595,None,...,Three Forks Dyke District,None,None,None,None,None,None,None,None,"POLYGON Z ((-111.52827 45.90741 0.00000, -111...."
2,3,1806000066,None,East side Madison River North of I-90,None,NaN,None,0.000855,0.139222,None,...,Madison Dyke and Drain District,None,None,None,None,None,None,None,None,"POLYGON Z ((-111.51480 45.91055 0.00000, -111...."
3,4,1806000065,None,East side Madison River South of I-90,None,NaN,None,0.006034,0.567880,None,...,Madison Dyke and Drain District,None,None,None,None,None,None,None,None,"POLYGON Z ((-111.44554 45.88636 0.00000, -111...."
4,6,1806000063,None,Town of Ekalaka,None,NaN,None,0.000028,0.023839,None,...,Carter County,None,None,None,None,None,None,None,None,"POLYGON Z ((-104.54174 45.88921 0.00000, -104...."


In [24]:
clip_possible_gdf = features_gdf.iloc[features_gdf.sindex.query(boundary_gdf.iloc[0].geometry)]
clip_possible_gdf.shape

(671, 51)

In [25]:
clip_exact_gdf = clip_possible_gdf[clip_possible_gdf.geometry.within(boundary_gdf.iloc[0].geometry)]
clip_exact_gdf.shape

(343, 51)

In [28]:
clipped_folder = os.path.abspath("./clipped")
!mkdir -p {clipped_folder}
clip_exact_gdf.to_file(os.path.join(clipped_folder, "clipped.shp"))
!ls {clipped_folder} -alh

/home/jovyan/work/mycondaenvs/test/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


total 12M
drwxr-xr-x 2 jovyan users 4.0K Mar 11 22:24 .
drwxr-xr-x 8 jovyan users 4.0K Mar 11 22:24 ..
-rw-r--r-- 1 jovyan users   10 Mar 11 22:24 clipped.cpg
-rw-r--r-- 1 jovyan users 1.3M Mar 11 22:24 clipped.dbf
-rw-r--r-- 1 jovyan users  145 Mar 11 22:24 clipped.prj
-rw-r--r-- 1 jovyan users  11M Mar 11 22:24 clipped.shp
-rw-r--r-- 1 jovyan users 2.8K Mar 11 22:24 clipped.shx


In [ ]:
# import shapely
# def row_func(row):
#     return shapely.wkb.loads(
#         shapely.wkb.dumps(row.geometry, output_dimension=2)).simplify(0.1)

# gdf_extact["geometry"] = gdf_extact.apply(row_func, axis=1)
# gdf_extact.shape

In [27]:
m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap)

features_data = GeoData(geo_dataframe = clip_exact_gdf,
                   style={'color': 'black', 'fillColor': 'red', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.8},
                   hover_style={'fillColor': 'yellow' , 'fillOpacity': 1},
                   name = 'Boundary')

boundary_data = GeoData(geo_dataframe = boundary_gdf,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':1, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.2},
                   name = 'Features')
m.fit_bounds([[boundary_gdf.bounds.miny[0], boundary_gdf.bounds.minx[0]], [boundary_gdf.bounds.maxy[0], boundary_gdf.bounds.maxx[0]]])
testarea = Textarea( value='Click on a feature to see attributes',
    placeholder='Type something',
    description='String:',
    disabled=True)
control = WidgetControl(widget=testarea, position='topright')
m.add_control(control)
def update_html(feature, **kwargs):
     testarea.value = '''
     {}
      '''.format(json.dumps(feature['properties'], indent=1))
features_data.on_click(update_html)

m.add_layer(boundary_data)
m.add_layer(features_data)
m.add_control(LayersControl())
m

Map(center=[52.3, 8.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…